In [2]:
import pandas as pd
from json_functions import create_df_simple, create_example

# players_list, df = create_df('test.json')

players_list, df = create_df_simple('very-big.json')

In [22]:
import numpy as np
import tensorflow as tf

In [23]:
features, outputs = df["rosters vector"], df['rating vector']

features = pd.DataFrame(features.values.tolist(), index= df.index)
outputs = pd.DataFrame(outputs.values.tolist(), index= df.index)

print(np.shape(features))
print(np.shape(outputs))


(8744, 1068)
(8744, 10)


In [24]:
import tensorflow as tf

In [25]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [26]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(1068, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	# model.add(Dense(50, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

In [28]:
n_inputs, n_outputs = features.shape[1], outputs.shape[1]

In [91]:
model = get_model(n_inputs, n_outputs)

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.33, random_state=42)

In [93]:
model.fit(X_train, y_train, verbose=1, epochs=4, batch_size=500)
print("Evaluation on test set:")
mae = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/4
12/12 [==============================] - 0s 6ms/step - loss: 0.7668
Epoch 2/4
12/12 [==============================] - 0s 7ms/step - loss: 0.2840
Epoch 3/4
12/12 [==============================] - 0s 6ms/step - loss: 0.2137
Epoch 4/4
12/12 [==============================] - 0s 6ms/step - loss: 0.1888
Evaluation on test set:
91/91 [==============================] - 0s 2ms/step - loss: 0.1961


In [94]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 1068)              1141692   
_________________________________________________________________
dense_30 (Dense)             (None, 10)                10690     
Total params: 1,152,382
Trainable params: 1,152,382
Non-trainable params: 0
_________________________________________________________________


In [95]:
example = create_example(['EliGE', 'Stewie2K', 'Grim', 'FalleN', 'NAF'], ['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], players_list)
model.predict(np.asarray([example]))

array([[1.1279236 , 1.1067467 , 0.796592  , 0.68875617, 0.7446804 ,
        1.5647084 , 1.1852388 , 1.123609  , 0.97567785, 0.88403565]],
      dtype=float32)

In [118]:
def print_prediction(example, players_list, model):
    player_indices = [i for i, x in enumerate(example) if x == 1]
    example_formatted = np.asarray([example])
    pred = model.predict(example_formatted)
    print("Predicted rating vector:", pred[0])

    print("Team 1:")
    for player in player_indices[0:5]:
        print(players_list[player]+', ', end='')
    print("\nRating spread:")
    for i in range(0, 5):
        print(round(pred[0][i], 2))
    print("Agregate Rating:", round(np.sum(pred[0][0:5]), 2))
    print("Team 2:")
    for player in player_indices[5:10]:
        print(players_list[player-len(players_list)]+', ', end='')
    print("\nRating spread:")
    for i in range(5, 10):
        print(round(pred[0][i], 2))
    print("Agregate Rating:", round(np.sum(pred[0][5:10]), 2))



In [119]:
# This model unfortunately isn't able to predict rating by player, just the spread of rating for the team.
print_prediction(example, players_list, model)

Predicted rating vector: [1.1279236  1.1067467  0.796592   0.68875617 0.7446804  1.5647084
 1.1852388  1.123609   0.97567785 0.88403565]
Team 1:
EliGE, NAF, Stewie2K, FalleN, Grim, 
Rating spread:
1.13
1.11
0.8
0.69
0.74
Agregate Rating: 4.46
Team 2:
device, dupreeh, Xyp9x, gla1ve, Magisk, 
Rating spread:
1.56
1.19
1.12
0.98
0.88
Agregate Rating: 5.73


Compare these results to a couple real matches played between these two teams on March 26th that the model hasn't seen:

https://www.hltv.org/stats/matches/mapstatsid/117239/liquid-vs-astralis?rankingFilter=Top10

https://www.hltv.org/stats/matches/mapstatsid/117231/astralis-vs-liquid?rankingFilter=Top10

Here's a hypothetical game that could never happen: 2016 Astralis vs current Astralis. Current Astralis is the most dominant CSGO roster of all time, so we would expect to see higher ratings accross the board for them, which we do. (They are team 2 here).

In [120]:
example2 = create_example(['device', 'Xyp9x', 'Kjaerbye', 'dupreeh', 'karrigan'], ['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], players_list)
print_prediction(example2, players_list, model)

Predicted rating vector: [1.3163592  1.1530508  0.82083374 0.8201335  0.71751314 1.287491
 1.0751294  0.8909651  0.878684   0.74764585]
Team 1:
karrigan, device, dupreeh, Xyp9x, Kjaerbye, 
Rating spread:
1.32
1.15
0.82
0.82
0.72
Agregate Rating: 4.83
Team 2:
device, dupreeh, Xyp9x, gla1ve, Magisk, 
Rating spread:
1.29
1.08
0.89
0.88
0.75
Agregate Rating: 4.88


Lets try it with the teams switched now

In [121]:
example2 = create_example(['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], ['device', 'Xyp9x', 'Kjaerbye', 'dupreeh', 'karrigan'], players_list)
print_prediction(example2, players_list, model)

Predicted rating vector: [1.3001164  1.0605851  0.913777   0.8430965  0.77797097 1.1263063
 1.0598643  0.85265464 0.76967937 0.60221595]
Team 1:
device, dupreeh, Xyp9x, gla1ve, Magisk, 
Rating spread:
1.3
1.06
0.91
0.84
0.78
Agregate Rating: 4.9
Team 2:
karrigan, device, dupreeh, Xyp9x, Kjaerbye, 
Rating spread:
1.13
1.06
0.85
0.77
0.6
Agregate Rating: 4.41


In [122]:
# One of the most unbalanced possible teams
example2 = create_example(['device', 's1mple', 'NiKo', 'ZywOo', 'electronic'], ['daps', 'karrigan', 'Golden', 'TACO', 'AdreN'], players_list)
print_prediction(example2, players_list, model)

Predicted rating vector: [1.5245003  1.1931746  1.0288424  0.8088182  0.8031617  1.1591073
 0.87344927 0.9006876  0.7326442  0.7448344 ]
Team 1:
s1mple, electronic, NiKo, device, ZywOo, 
Rating spread:
1.52
1.19
1.03
0.81
0.8
Agregate Rating: 5.36
Team 2:
karrigan, Golden, AdreN, TACO, daps, 
Rating spread:
1.16
0.87
0.9
0.73
0.74
Agregate Rating: 4.41


In [123]:
# Astralis vs a random team of players that generally get low rating
example2 = create_example(['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], ['daps', 'karrigan', 'Golden', 'TACO', 'AdreN'], players_list)
print_prediction(example2, players_list, model)

Predicted rating vector: [1.6451367  1.2084872  1.1477096  1.0272878  0.8838766  1.1525126
 0.87884426 0.7957482  0.6789951  0.6662946 ]
Team 1:
device, dupreeh, Xyp9x, gla1ve, Magisk, 
Rating spread:
1.65
1.21
1.15
1.03
0.88
Agregate Rating: 5.91
Team 2:
karrigan, Golden, AdreN, TACO, daps, 
Rating spread:
1.15
0.88
0.8
0.68
0.67
Agregate Rating: 4.17


Since Astralis has more impressive ratings than the above super team against my "bad team", It's possible that model is capturing the chemistry that Astralis would have.

In [124]:
# Astralis vs All-Stars
example2 = create_example(['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], ['device', 's1mple', 'NiKo', 'ZywOo', 'electronic'], players_list)
print_prediction(example2, players_list, model)

Predicted rating vector: [1.3088439  1.0866596  0.96441    0.91596353 0.8278573  1.2931731
 1.0061758  0.8958529  0.7874176  0.63033223]
Team 1:
device, dupreeh, Xyp9x, gla1ve, Magisk, 
Rating spread:
1.31
1.09
0.96
0.92
0.83
Agregate Rating: 5.1
Team 2:
s1mple, electronic, NiKo, device, ZywOo, 
Rating spread:
1.29
1.01
0.9
0.79
0.63
Agregate Rating: 4.61


Since Astralis even beats the all-star team head to head, despite the all-star team having the 5 highest rated players in my entire dataset, I think this is further evidence that my model is capturing some information about player chemistry

In [125]:
model.save('model_01.h5')

In [3]:
df['rating vector']

0       [1.55, 1.47, 1.07, 0.88, 0.86, 1.32, 1.11, 1.0...
1       [1.75, 1.49, 1.32, 1.19, 0.99, 0.9, 0.88, 0.83...
2       [1.48, 1.28, 1.25, 1.19, 0.79, 1.32, 1.03, 0.9...
3       [1.29, 1.15, 1.14, 1.01, 1.01, 1.52, 1.27, 1.0...
4       [1.62, 1.38, 1.22, 0.94, 0.74, 1.36, 1.18, 0.9...
                              ...                        
8739    [1.64, 1.15, 1.06, 1.06, 0.88, 1.16, 1.07, 0.9...
8740    [0.91, 0.86, 0.6, 0.58, 0.34, 1.98, 1.72, 1.31...
8741    [2.09, 1.34, 1.24, 0.74, 0.71, 1.29, 1.08, 0.8...
8742    [1.48, 1.16, 1.14, 1.04, 0.77, 1.15, 0.94, 0.8...
8743    [1.42, 1.14, 1.09, 0.91, 0.8, 1.16, 1.11, 1.05...
Name: rating vector, Length: 8744, dtype: object